# Web Scraping Script

    In this section, we need to scrape images from e-commerce portal, Amazon.com. The clothing categories used for scraping will be:
-	Sarees (women)
-	Trousers (men)
-	Jeans (men)


In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

Detailed Requirement:

    A minimum of 200 images for each of the three categories-  Sarees (women), Trousers (men), Jeans (men) need to be scraped from Amazon portal.  This data will then be used as input to build the deep learning model for Image Classification based on the three image categories.

In [3]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib

# All in same directory
DRIVER_PATH = 'chromedriver.exe'


def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)        
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    error_clicks = 0
    while (image_count < max_links_to_fetch) & (error_clicks < 30): # error clicks to stop when there are no more results to show by Google Images. You can tune the number
        scroll_to_end(wd)

        print('Starting search for Images')

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        for img in thumbnail_results[results_start:max_links_to_fetch]:
            # try to click every thumbnail such that we can get the real image behind it
            print("Total Errors till now:", error_clicks)
            try:
                print('Trying to Click the Image')
                img.click()
                time.sleep(sleep_between_interactions)
                print('Image Click Successful!')
            except Exception:
                error_clicks = error_clicks + 1
                print('ERROR: Unable to Click the Image')
                if(results_start < number_results):
                	continue
                else:
                	break
                	
            results_start = results_start + 1

            # extract image urls    
            print('Extracting of Image URLs')
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            print('Current Total Image Count:', image_count)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
            else:
                load_more_button = wd.find_element_by_css_selector(".mye4qd")
                if load_more_button:
                    wd.execute_script("document.querySelector('.mye4qd').click();")
            	        
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    queries = ["Womens Sarees", "Trousers for Men"]  #change your set of queries here
    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = fetch_image_urls(query,200,wd) # 200 denotes no. of images you want to download
        images_path = 'dataset/'
        for i in links:
            persist_image(images_path,query,i)
    wd.quit()
    
    


Starting search for Images
Found: 100 search results. Extracting links from 0:100
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 1
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 2
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 3
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 4
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 5
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 6
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 7
Total Errors ti

ERROR: Unable to Click the Image
Total Errors till now: 6
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 102
Total Errors till now: 6
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 105
Total Errors till now: 6
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 107
Total Errors till now: 6
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 109
Total Errors till now: 6
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 111
Total Errors till now: 6
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 113
Total Errors till now: 6
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 115
Total Errors till now: 6
Trying to Click the Image

SUCCESS - saved https://sslimages.shoppersstop.com/B8AC9759D45547D9AEF177F0DE13B7C8/img/E29578EDA9A249E3B951666D4DF38A86/203260273_9607_E29578EDA9A249E3B951666D4DF38A86.jpg - as dataset/Womens Sarees\7e1bc7f1a4.jpg
SUCCESS - saved https://rukminim1.flixcart.com/image/332/398/kkk1vgw0/sari/i/7/2/free-bombey-01-light-pink-karkar-unstitched-original-imafzva8bzzteatc.jpeg?q=50 - as dataset/Womens Sarees\0c76143780.jpg
SUCCESS - saved https://assets.ajio.com/medias/sys_master/root/20210607/qfnI/60bd8c86aeb269a9e3d9b207/ethnic_junction_pink_kanjeevaram_silk_woven_half_&_half_saree.jpg - as dataset/Womens Sarees\e1925e4aa1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRdADKp5rflCdxeXWaVWqpCkroCl2m38AUF_g&usqp=CAU - as dataset/Womens Sarees\e6b111ffd6.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRPcIUK6WqJnirTTggw6G4QT0jnsmpbKzUifA&usqp=CAU - as dataset/Womens Sarees\796bfeb787.jpg
SUCCESS - saved https://imgshopimages.lbb.in/catalog/pro

SUCCESS - saved https://5.imimg.com/data5/ANDROID/Default/2021/5/LB/PA/UX/129118599/product-jpeg-500x500.jpg - as dataset/Womens Sarees\244b06024e.jpg
SUCCESS - saved https://fcity.in/images/products/4009557/2_512.jpg - as dataset/Womens Sarees\c4ac973ce2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSn0EjK_swGCcuEHhJIdPHz4YaB_vNDMmzI1Q&usqp=CAU - as dataset/Womens Sarees\fad1a80b35.jpg
SUCCESS - saved https://assetscdn1.paytm.com/images/catalog/product/A/AP/APPPACHIBA-ENTEPACH1154748A825397F/1581491977107_0..jpg - as dataset/Womens Sarees\daf9038383.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSAH7UBhQTCC2Ov4vL0h6sWtFQDrEBuVFl9fQ&usqp=CAU - as dataset/Womens Sarees\ad5a8cd514.jpg
SUCCESS - saved https://www.wholesaletextile.in/product-img/Ranjna-Retro-Women-s-Sarees-On-1618226642.jpg - as dataset/Womens Sarees\b1eeacb67c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQnRxz4MiMViYRNz1Es2XjQ60Mix5-USL2Zm

SUCCESS - saved https://cdn.fashiola.in/L68295551/rajnandini-women-sarees-women-blue-white-warli-print-saree.jpg - as dataset/Womens Sarees\8170701b49.jpg
SUCCESS - saved https://n2.sdlcdn.com/imgs/j/t/b/230X258_sharpened/Bhakarwadi-Pink-Khadi-Saree-SDL097991163-1-082df.jpeg - as dataset/Womens Sarees\3bdfc21f17.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQuyVI9RYA4jqtGPJRFTv1hncCQdr8RMQ8L1g&usqp=CAU - as dataset/Womens Sarees\91f1a91d3f.jpg
SUCCESS - saved https://www.bigbindi.com/media/catalog/product/cache/6/small_image/295x406/9df78eab33525d08d6e5fb8d27136e95/R/T/RT-117518.jpg - as dataset/Womens Sarees\eedc7c6c62.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ0V77jvnkCd19cPdulKEy8cEXsQ9P2BhWpBw&usqp=CAU - as dataset/Womens Sarees\a24477cfcb.jpg
SUCCESS - saved https://img3.junaroad.com/uiproducts/17551329/pri_175_p-1613468382.jpg - as dataset/Womens Sarees\7fe8417d61.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/im

...

Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 129
Total Errors till now: 2
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 131
Total Errors till now: 2
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 133
Total Errors till now: 2
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 135
Total Errors till now: 2
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 137
Total Errors till now: 2
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 139
Total Errors till now: 2
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 141
Total Errors till now: 2
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 142
Tot

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQzWMTCexenic0DcEflsUewZ-9qjWrR-jRxRA&usqp=CAU - as dataset/Trousers for Men\c80d63db4a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSW7IK4qHB-zXUY-SXHgZrRBJBv7IxmP35k_w&usqp=CAU - as dataset/Trousers for Men\1980f8b995.jpg
SUCCESS - saved https://cdn.shopclues.com/images1/thumbnails/104183/320/320/148659823-104183285-1579955777.jpg - as dataset/Trousers for Men\75a65ba762.jpg
SUCCESS - saved https://image.made-in-china.com/2f0j00SCZGsOqrEpkt/Men-s-Trousers-Casual-Ninth-Pants-Men-s-Trend-Wild-Loose-Sports-Pants-Ice-Silk-Summer-Thin-Section-Men-s-Korean-Style-Trousers.jpg - as dataset/Trousers for Men\10ce9be0dc.jpg
SUCCESS - saved https://assets.ajio.com/medias/sys_master/root/20210316/z5Dh/604fae0baeb269698186406d/-288Wx360H-441116324-ltgrey-MODEL.jpg - as dataset/Trousers for Men\abbf882f7f.jpg
ERROR - Could not save https://alitools.io/en/showcase/image?url=https%3A%2F%2Fae01.alicdn.com

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQFuDAY38DpH_dnF9ua8Z81ypJhfxMnvLZMAg&usqp=CAU - as dataset/Trousers for Men\7cd4a06bbe.jpg
SUCCESS - saved https://cdn.shopify.com/s/files/1/1231/6442/products/B0630-COTTONWORLD-198LOT-26STYLES6026_394x.jpg?v=1614067518 - as dataset/Trousers for Men\07aa4ea55e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSJzvEdH0tqQ0ciXLx_-YTbZkXAnjKyWAfgFA&usqp=CAU - as dataset/Trousers for Men\3d67fc182a.jpg
SUCCESS - saved https://cdn.fashiola.in/L59667071/dennison-men-trousers-men-off-white-grey-smart-tapered-fit-self-checked-regular-trousers.jpg - as dataset/Trousers for Men\2578d6947d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRVuJXZIbdFt0U1W2BDzQvXRyjhjK62GXlbpA&usqp=CAU - as dataset/Trousers for Men\d88934d711.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRcmz05c8u1NkTQg-3dF3pIm0DKduFmE9kh8g&usqp=CAU - as dataset/Trousers for Men\c4cfbac56

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTG_ockKdkDFLwUVS3lWvRPkemKLb6rxVyA5g&usqp=CAU - as dataset/Trousers for Men\72bd8d03a2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSj_0AQ6cE2_4KYmxX609q10NM51uzTSmQshQ&usqp=CAU - as dataset/Trousers for Men\9c072d92db.jpg
SUCCESS - saved https://cf.fgroup.workers.dev/bf/resize?key=300x400/50012522639NAVY_default_super_zoom_image_48a376879f65433aaedfd121aa3d6fca.jpg - as dataset/Trousers for Men\0b59e46509.jpg
SUCCESS - saved https://4.imimg.com/data4/JN/UI/MY-28459816/trousers-1-500x500.jpg - as dataset/Trousers for Men\4c3003c51b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQOK5ixxpWXTpT-mrFtmkfJkqKDigIYSvuN2w&usqp=CAU - as dataset/Trousers for Men\0528d50b2d.jpg
SUCCESS - saved https://i.pinimg.com/736x/16/f9/ec/16f9ec9f31eec24d924f1b12f3adc592.jpg - as dataset/Trousers for Men\5b16978645.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:A

SUCCESS - saved https://adn-static1.nykaa.com/nykdesignstudio-images/tr:w-410,/pub/media/catalog/product/a/f/afztr2062_1_9dcbe787.jpg?rnd=20200526195200 - as dataset/Trousers for Men\f41c9d248b.jpg
SUCCESS - saved https://www.7camicie.com/15297-big_default/trousers-men-chinos-twill-light-brown-7229.jpg - as dataset/Trousers for Men\b92db6bc54.jpg
SUCCESS - saved https://img1.exportersindia.com/product_images/bc-full/dir_11/305332/mens-cotton-trousers-1496993.jpg - as dataset/Trousers for Men\87ef97b704.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR87yYMMFlZzbBb5X94K-aQwo_jEPv3zwnNWw&usqp=CAU - as dataset/Trousers for Men\6ac53ff9d3.jpg
SUCCESS - saved https://pantaloons.imgix.net/img/app/product/4/482297-3519511.jpg?w=618&auto=format - as dataset/Trousers for Men\4b7a5aea4d.jpg
SUCCESS - saved https://assetscdn1.paytm.com/images/catalog/product/A/AP/APPEDITLOOK-MENVAIB73553974554F7A/1599634177191_0..jpg - as dataset/Trousers for Men\94f283e709.jpg
SUCCESS -

In [4]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib

# All in same directory
DRIVER_PATH = 'chromedriver.exe'


def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)        
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    error_clicks = 0
    while (image_count < max_links_to_fetch) & (error_clicks < 30): # error clicks to stop when there are no more results to show by Google Images. You can tune the number
        scroll_to_end(wd)

        print('Starting search for Images')

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        for img in thumbnail_results[results_start:max_links_to_fetch]:
            # try to click every thumbnail such that we can get the real image behind it
            print("Total Errors till now:", error_clicks)
            try:
                print('Trying to Click the Image')
                img.click()
                time.sleep(sleep_between_interactions)
                print('Image Click Successful!')
            except Exception:
                error_clicks = error_clicks + 1
                print('ERROR: Unable to Click the Image')
                if(results_start < number_results):
                	continue
                else:
                	break
                	
            results_start = results_start + 1

            # extract image urls    
            print('Extracting of Image URLs')
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            print('Current Total Image Count:', image_count)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
            else:
                load_more_button = wd.find_element_by_css_selector(".mye4qd")
                if load_more_button:
                    wd.execute_script("document.querySelector('.mye4qd').click();")
            	        
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    queries = ['Mens Jeans']  #change your set of queries here
    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = fetch_image_urls(query,200,wd) # 200 denotes no. of images you want to download
        images_path = 'dataset/'
        for i in links:
            persist_image(images_path,query,i)
    wd.quit()
    
    


Starting search for Images
Found: 100 search results. Extracting links from 0:100
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 1
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 2
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 2
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 2
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 3
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 4
Total Errors till now: 0
Trying to Click the Image
ERROR: Unable to Click the Image
Total Errors till now: 1
Trying to Click the Image
Image Cli

Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 100
Total Errors till now: 1
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 102
Total Errors till now: 1
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 104
Total Errors till now: 1
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 106
Total Errors till now: 1
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 108
Total Errors till now: 1
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 110
Total Errors till now: 1
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 111
Total Errors till now: 1
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 113
Tot

SUCCESS - saved https://image.made-in-china.com/44f3j00wSzQRMbrfdkq/Wholesale-Best-Selling-Fashion-Jeans-Pants-Mens-Jeans-Vintage-Jeans-Men.jpg - as dataset/Mens Jeans\8934e3732d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT8TDhxiHpR_azXf1zQF2FYol5TJsu4JmZT-A&usqp=CAU - as dataset/Mens Jeans\7d40dee1c5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQEoNCPRLO2pAHhP92Kxl5aqeV9viCGT_KQKw&usqp=CAU - as dataset/Mens Jeans\ee5d1c81bf.jpg
SUCCESS - saved https://redbridgejeans.de/media/image/product/65946/lg/m4253_red-bridge-mens-jeans-pants-slim-fit-denim-stonewashed-arena-b-m4253.jpg - as dataset/Mens Jeans\8370b1dbb8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTOW2IAVMTg9vOW-k_Rwt7RHzHWSXVTa5UyqQ&usqp=CAU - as dataset/Mens Jeans\234d33b7ae.jpg
SUCCESS - saved https://i.insider.com/5cae2a77c57fa6030b5cb6cb?width=700 - as dataset/Mens Jeans\eef6472e5a.jpg
SUCCESS - saved https://lsco.scene7.com/is/image/ls

C:\Users\Neha Vishnudas Pai\Documents\DATATRAINED\FOLDER\lib\site-packages\PIL\Image.py:2929: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


ERROR - Could not save https://www.heddels.com/wp-content/uploads/2011/06/MensJeans2.jpg - cannot identify image file <_io.BytesIO object at 0x000001BF4FC38AE0>
SUCCESS - saved https://assetscdn1.paytm.com/images/catalog/product/A/AP/APPMASTERLY-WEFDESI356484D2D7843/0..jpg - as dataset/Mens Jeans\acaedb4dfc.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQi0p0Ex6TZvHAoofWMdoHpVqTPmxQgs77bEg&usqp=CAU - as dataset/Mens Jeans\6ffb010a86.jpg
SUCCESS - saved https://i.insider.com/60b7f188bee0fc0019d5af9a?width=700 - as dataset/Mens Jeans\a73d759655.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRNt62yKCTwfOOAWVSj_mcC99_7MMaPFc6EIw&usqp=CAU - as dataset/Mens Jeans\4c07465183.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQp4ulw0XOMGe5DO1Se2empBnc5q7uXHhrFQg&usqp=CAU - as dataset/Mens Jeans\c79e8aecb4.jpg
SUCCESS - saved https://cdn.lifestyleasia.com/wp-content/uploads/2019/11/22150217/Best-mens-denim-HERO.jpg - a

SUCCESS - saved https://media.frenchconnection.com/ms/fcuk/54atv-mens-cl-washedgrey-denim-slim-fit-jeans.jpg?404=fcuk/54atv.jpg&width=0&height=768 - as dataset/Mens Jeans\8de47d3644.jpg
SUCCESS - saved https://pantaloons.imgix.net/img/app/product/5/524960-3956735.jpg - as dataset/Mens Jeans\7ea0eb39da.jpg
SUCCESS - saved https://pyxis.nymag.com/v1/imgs/436/b06/3b0e7863b096e15e6c4ec9523658f27898-PhotoSend.2x.rdeep-vertical.w245.jpg - as dataset/Mens Jeans\912e80a957.jpg
SUCCESS - saved https://images.bewakoof.com/t320/high-star-mens-blue-washed-slim-fit-mid-rise-clen-look-light-faded-jeans-with-belt-loops-mbdrjk690-high-star-men-s-black-washed-casual-jeans-342635-1615889754.jpg - as dataset/Mens Jeans\aa083c9729.jpg
SUCCESS - saved https://www.express.com/content/dam/express/2021/projects/web/category-inline/08-august/0809-floorset/0809-digital-11633-m-jeans/fits/poster-images/05-m-jean-nav-slim.jpg - as dataset/Mens Jeans\b084f7eb02.jpg
SUCCESS - saved https://cdn11.bigcommerce.com/s-p

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRtr3AcUXcrgV_QPL6qiLuhiG8A0G0L4j66zw&usqp=CAU - as dataset/Mens Jeans\fb8337c770.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSAbzQdT_1FlNHuKXLyg-T3Wc1ob3rdGNMHyw&usqp=CAU - as dataset/Mens Jeans\3423285307.jpg
SUCCESS - saved https://m.media-amazon.com/images/I/71erJErmaQS._UL1500_.jpg - as dataset/Mens Jeans\3a791e9903.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRuVJcXXLZdRJxArUZ9_b-5TuJ-YvRAS2Ixtg&usqp=CAU - as dataset/Mens Jeans\0a75a39487.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRW2jJ-jcM-6APYQPkOf2zoDVipDRuhK0EiSg&usqp=CAU - as dataset/Mens Jeans\c5581cffca.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTUGLPj0iBMgQhb2jeFDHHXD2-TEyqG4XIsOg&usqp=CAU - as dataset/Mens Jeans\e00ad1fa1d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQM0dmXnYXhRxYpknyUFtChvGIdEsStXxujcw&us

SUCCESS - saved https://assetscdn1.paytm.com/images/catalog/product/A/AP/APPSPARTACUS2FABANS11736131C5024C7/1617427080269_0..jpg - as dataset/Mens Jeans\70e94ddfa3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ_h87QeMejQcmN8SlGX8vzEuyvZWJu-N17Gw&usqp=CAU - as dataset/Mens Jeans\cddefb860a.jpg
SUCCESS - saved https://ashleyweston.com/wp-content/uploads/2015/10/How-Your-Jeans-Should-Fit-Ashley-Weston-Mens-Clothing-Fit-Guide.jpg - as dataset/Mens Jeans\6299cf60ac.jpg
